In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df = df.loc[df['type']!=0]
df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
df['aid2'] = df.aid
print( df.shape )
df.head()

(725300, 7)


,session,aid,ts,type,hour,day,aid2
7,11098530,409236,1661120532,1,80532,339732,409236
28,11098531,1728212,1661119746,2,79746,338946,1728212
29,11098531,452188,1661119746,2,79746,338946,452188
30,11098531,1271998,1661119746,2,79746,338946,1271998
31,11098531,396199,1661119746,2,79746,338946,396199


In [3]:
user_features = df.groupby('session').agg({'type':'mean','aid':'count','aid2':'nunique','hour':'mean','day':'mean'})
user_features.columns = ['buy_ratio2','count_item2','unique_item2','hour_mean2','day_mean2']

In [4]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2
session,,,,,
11098530,1.0,1,1,80532.0,339732.0
11098531,2.0,4,4,79746.0,338946.0
11098533,1.0,2,2,79339.5,338539.5
11098535,1.0,1,1,79263.0,338463.0
11098537,1.4,5,2,82032.8,341232.8


In [5]:
user_features2 = df.groupby('session').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
user_features2.columns = ['buy_ratio_std2','hour_std2','day_std2']

f32 = ['buy_ratio_std2','hour_std2','day_std2']
for c in f32: user_features2[c] = user_features2[c].astype('float32')

In [6]:
user_features2.head()

,buy_ratio_std2,hour_std2,day_std2
session,,,
12260713,0.447214,742.730286,742.730286
11995080,0.000000,8843.485352,8843.485352
11847637,-1.000000,-1.000000,-1.000000
12293367,-1.000000,-1.000000,-1.000000
12057855,0.000000,113.182747,113.182747


In [7]:
user_features['repeat2'] = user_features.count_item2 / user_features.unique_item2

f32 = ['buy_ratio2','hour_mean2','day_mean2','repeat2']
for c in f32: user_features[c] = user_features[c].astype('float32')
    
i32 = ['count_item2','unique_item2']
for c in i32: user_features[c] = user_features[c].astype('int32')

In [8]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [10]:
user_features.columns = [x.replace('2','10') for x in user_features.columns]

In [11]:
user_features.head()

,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
session,,,,,,,,,
11098530,1.0,1,1,80532.000000,339732.0000,1.0,-1.000000,-1.000000,-1.000000
11098531,2.0,4,4,79746.000000,338946.0000,1.0,0.000000,0.000000,0.000000
11098533,1.0,2,2,79339.500000,338539.5000,1.0,0.000000,195.868576,195.868576
11098535,1.0,1,1,79263.000000,338463.0000,1.0,-1.000000,-1.000000,-1.000000
11098537,1.4,5,2,82032.796875,341232.8125,2.5,0.547723,598.668274,598.668274


In [12]:
user_features.dtypes

buy_ratio10        float32
count_item10         int32
unique_item10        int32
hour_mean10        float32
day_mean10         float32
repeat10           float32
buy_ratio_std10    float32
hour_std10         float32
day_std10          float32
dtype: object

In [13]:
user_features.to_parquet('../../data/item_user_features/user10.pqt')